<a href="https://colab.research.google.com/github/paolofantozzi/codemotion2020/blob/main/Beatles_scraping_and_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [2]:
from pathlib import Path
project_path = Path('drive/My Drive/colab_data/beatles')

In [3]:
!pip install lyricsgenius

     |████████████████████████████████| 51kB 5.5MB/s 


In [4]:
import lyricsgenius

genius = lyricsgenius.Genius('viAbQje3x3nPl-ReFJMT_s10V00XIHmhcGLtiqn73kSGh0ruipkg60_dyLf6ibfc')
genius.skip_non_songs = True
genius.excluded_terms = ['(Remix)', '(Live)', '(Take)', '(Version)', '(Mix)', '(Demo)']

lyrics_path = project_path / 'Lyrics_TheBeatles.json'
if not lyrics_path.exists():
  artist = genius.search_artist('Beatles', sort='title')
  artist.save_lyrics(filename=str(lyrics_path))

In [5]:
import json

def is_lyric(text):
  if not text:
    return False
  if '[Verse' not in text and '[Chorus]' not in text:
    return False
  return True

with open(lyrics_path, 'r') as lyrics_file:
  artist = json.load(lyrics_file)
  lyrics = [{'title': s['title'], 'lyrics': s['lyrics']} for s in artist['songs'] if is_lyric(s['lyrics'])]

discarded = len(artist['songs']) - len(lyrics)
print(f'Discarded {discarded}')
print(f'Keeped {len(lyrics)}')


Discarded 136
Keeped 268


In [6]:
def pre_process_song(song):
  song_title = song['title']
  song_lyrics = song['lyrics']
  return f'<s_song>\n{song_title}\n[Lyrics]\n{song_lyrics}\n<e_song>\n'

In [7]:
from sklearn.model_selection import train_test_split

all_data = [pre_process_song(song) for song in lyrics]
train_data_str, test_data_str = train_test_split(all_data, test_size=0.2)

print(f'Train size: {len(train_data_str)}')
print(f'Test size: {len(test_data_str)}')

Train size: 214
Test size: 54


In [8]:
train_path = project_path / 'train_data.txt'
with open(train_path, 'w') as train_data_file:
  train_data_file.writelines(f'{line}\n' for line in train_data_str)

test_path = project_path / 'test_data.txt'
with open(test_path, 'w') as test_data_file:
  test_data_file.writelines(f'{line}\n' for line in test_data_str)

In [9]:
!pip install transformers

     |████████████████████████████████| 1.3MB 11.3MB/s 
     |████████████████████████████████| 1.1MB 49.7MB/s 
     |████████████████████████████████| 2.9MB 52.3MB/s 
     |████████████████████████████████| 890kB 48.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=4707d245f733b9b1946af80f33f24379e45191cd286782ea97f14154dc408d17
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [10]:
from transformers import AutoModelWithLMHead
from transformers import AutoTokenizer

MODEL_NAME = 'gpt2'
model = AutoModelWithLMHead.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.6/dist-packages/transformers/modeling_auto.py:825: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [11]:
from transformers import DataCollatorForLanguageModeling
from transformers import TextDataset

train_data = TextDataset(
    tokenizer=tokenizer,
    file_path=str(train_path),
    block_size=128,
)

test_data = TextDataset(
    tokenizer=tokenizer,
    file_path=str(test_path),
    block_size=128,
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

In [12]:
from transformers import Trainer
from transformers import TrainingArguments

model_path = Path('./gpt2-beatles')
logs_path = Path('./logs')

training_args = TrainingArguments(
  output_dir=str(model_path),     # output directory
  logging_dir=str(logs_path),     # logs directory
  overwrite_output_dir=True,      # overwrite the content of the output directory
  num_train_epochs=70,            # number of training epochs
  per_device_train_batch_size=16, # batch size for training
  per_device_eval_batch_size=32,  # batch size for evaluation
  eval_steps=400,                 # number of update steps between two evaluations
  save_steps=800,                 # after how many steps model is saved
  warmup_steps=500,               # number of warmup steps for learning rate scheduler
)

trainer = Trainer(
  model=model,
  args=training_args,
  data_collator=data_collator,
  train_dataset=train_data,
  eval_dataset=test_data,
  prediction_loss_only=True,
)

/usr/local/lib/python3.6/dist-packages/transformers/trainer.py:263: FutureWarning: Passing `prediction_loss_only` as a keyword argument is deprecated and won't be possible in a future version. Use `args.prediction_loss_only` instead. Setting `args.prediction_loss_only=True
  FutureWarning,


In [13]:
trainer.train()

Step,Training Loss
500,2.166332
1000,0.956241
1500,0.357635
2000,0.188342
2500,0.138242


TrainOutput(global_step=2520, training_loss=0.7563310895647322)



```
function ClickConnect(){document.querySelector("#comments").click();}
setInterval(ClickConnect,60000)
```



In [14]:
trainer.evaluate()

{'epoch': 70.0, 'eval_loss': 4.07750940322876, 'total_flos': 3766404735959040}

In [15]:
trainer.save_model()

In [23]:
from transformers import pipeline

def generate(title, temperature, top_k):
  generator = pipeline('text-generation', model=str(model_path), tokenizer=tokenizer)
  return generator(
        f'<s_song>\n{title}\n[Lyrics]\n',
        max_length=10**3,
        temperature=temperature,
        top_k=top_k
  )[0]['generated_text']

In [17]:
print(generate('Love of my life', temperature=0.8, top_k=0))

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


<s_song>
Love of my life
[Lyrics]
[Verse 1]
Nothing can come between us
When it gets dark I tow your heart away
Never thought of act of love I've ever known

[Chorus]
For if my love had lasted a day
We might have broken down
If we'd seen each other before

[Verse 2]
I can't forget the time or place
Where we just met, she's just the girl for me
And I want all the world to see we've met
Mmm-mmm-mmm-da-da

[Chorus]
Love of my life
[Verse 3]
Nothing can come between us
When it gets dark I tow your heart away
Never thought of act of love I've ever known

[Chorus]
For if my love had lasted a day
We might have broken down
If we'd seen each other before

[Verse 4]
I can't forget the time or place
Where we just met, she's just the girl for me
And I want all the world to see we've met
Mmm-mmm-da-da-da

[Chorus]
Love of my life
[Verse 5]
I can't forget the time or place
Where we just met, she's just the girl for me
And I want all the world to see we've met
Mmm-mmm-da-da-da

[Chorus]
For if my lov

In [19]:
print(generate('Love of my life', temperature=0.8, top_k=40))

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


<s_song>
Love of my life
[Lyrics]
[Verse 1]
There is one love I have
Love that keeps me on my toes
Sitting here in Blue Jay Way

[Verse 2]
Don't want to spend
Another day here
Loving you like I've never done
In the morning
Just waiting for you

[Chorus]
Love of my life
Love of my life
Love of my life

[Verse 3]
Longer than the road that stretches out ahead

[Bridge]
And before too long, I fall in love with you

[Instrumental Break]

[Chorus]

[Verse 1]
Don't want to spend
Another day here
Loving you like I've never done
In the morning
Just waiting for you

[Chorus]

[Verse 2]
Don't want to spend
Another day here
Loving you like I've never done
In the morning
Just waiting for you

[Chorus]

[Verse 3]
Love of my life
Love of my life
Love of my life

[Bridge]
And before too long, I fall in love with you

[Instrumental Break]

[Chorus]

[Verse 1]
Don't want to spend
Another day here
Loving you like I've never done
In the morning
Just waiting for you

[Chorus]

[Verse 2]
Don't want to spend

In [20]:
print(generate('Love of my life', temperature=1.2, top_k=0))

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


<s_song>
Love of my life
[Lyrics]
[Verse 1]
It is not enough for me to show you my love
That is, I must show you my love
Who can forget the love I have who came to share
And show you the joy that comes from holding you close

[Verse 2]
When I call you on the telephone
You say, "It's me"
When I whisper in your ear
You say, "It's me, love"
Just like I've known the secret for some time now

[Chorus]
Now this is the end, happy with you

[Outro]
Love of my life
Love of my life
Love of my life

[Verse 3]
When I write down a short list of love songs
Each one speaking of a love that has already been
And leaving no stone unturned
I just have to suppose it belongs to you

[Chorus]
All the songs that you've just heard
All the songs that you're thinking of
All the songs that you're thinking of
All the songs that you're thinking of
All the songs that you're thinking of
All the songs that you're thinking of
All the songs that you're thinking of
All the songs that you're thinking of
All the songs tha

In [24]:
print(generate('Love of my life', temperature=1.2, top_k=40))

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


<s_song>
Love of my life
[Lyrics]
[Verse 1]
The stars that shine above
Will light our way to love

[Chorus]
Love of my life
Love of my life
Love of my life

[Verse 2]
In the evening sky
People tell me that there's someone there
But I haven't seen him there

[Chorus]
There's nowhere I can be
Just like this little blue hilltop
Will I ever be free
So go away

[Instrumental Verses]

[Verse 3]
As the June light turns to moonlight
I tune in to the sound
Like an octopus on an electric light dance

[Chorus]
As the June light turns to moonlight
I tune in to the sound
Like an octopus on an electric light dance

[Chorus]
I know I never really been aware of this before
But I've learned to dance like an African dancer

[Instrumental Verses]

[Verse 4]
At night where the sun is shining
People tell me that there's a presence there
But I haven't seen him there

[Chorus]
There's nowhere I can be
Just like this little blue hilltop
Will I ever be free
So go away

[Instrumental Verses]

[Verse 5]
As the J